# Adding chairpersons to NFDI consortia
Our goal here is adding information about chairpersons (spokespersons) of NFDI consortia to Wikidata.

In [1]:
import requests
import pandas as pd

Let's check previously parsed data.

In [2]:
df = pd.read_csv("../../../data/GEPRIS_NFDI_project_pages.csv")
df

,gepris,title,subject_area,homepage,description,Applying institution,Co-applicant institution,Participating Institution,Spokesperson,Participating Persons,Co-Spokespersons,Cooperation partners
0,https://gepris.dfg.de/gepris/projekt/441914366,GHGA,"Medicine, Biology",https://ghga.dkfz.de/,Human genome sequencing and other omics data m...,"[('Deutsches Krebsforschungszentrum (DKFZ)', '...","[('Charité - Universitätsmedizin Berlin', 'htt...",[('CISPA - Helmholtz-Zentrum für Informationss...,"[('Professor Dr. Oliver Stegle, Ph.D.', 'http...","[('Viktor Achter', 'https://gepris.dfg.de/gep...","[('Privatdozent Dr. Peer Bork', 'https://gepr...",NaN
1,https://gepris.dfg.de/gepris/projekt/441926934,NFDI4Cat,"Biology, Chemistry, Mathematics, Thermal Engin...",http://gecats.org/NFDI4Cat.html,The overall strategy for the transformation of...,[('DECHEMA Gesellschaft für Chemische Technik ...,[('Fraunhofer-Institut für Offene Kommunikatio...,"[('Technische Universität Darmstadt', 'https:/...","[('Dr. Andreas Förster', 'https://gepris.dfg....","[('Professor Dr.-Ing. Bastian Etzold', 'https...","[('Professor Dr. Matthias Beller', 'https://g...",NaN
2,https://gepris.dfg.de/gepris/projekt/441958017,NFDI4Culture,"Humanities, Construction Engineering and Archi...",https://nfdi4culture.de,Digital data on tangible and intangible cultur...,[('Akademie der Wissenschaften und der Literat...,[('FIZ KarlsruheLeibniz-Institut für Informati...,[('Arbeitsgemeinschaft der kunsthistorischen B...,"[('Professor Torsten Schrade', 'https://gepri...",[('Professorin Dr. Stefanie Acquavella-Rauch'...,"[('Reinhard Altenhöner', 'https://gepris.dfg....",NaN
3,https://gepris.dfg.de/gepris/projekt/441958208,NFDI4Chem,Chemistry,https://nfdi4chem.de,The vision of NFDI4Chem is the digitalisation ...,"[('Friedrich-Schiller-Universität Jena', 'http...",[('FIZ KarlsruheLeibniz-Institut für Informati...,[('Beilstein-Institut zur Förderung der chemis...,"[('Professor Dr. Christoph Steinbeck', 'https...","[('Privatdozent Dr. Carsten Baldauf', 'https:...","[('Dr. Felix Bach', 'https://gepris.dfg.de/ge...",NaN
4,https://gepris.dfg.de/gepris/projekt/442032008,NFDI4BioDiversity,"Biology, Medicine, Agriculture, Forestry and V...",https://www.nfdi4biodiversity.org,Biodiversity is more than just the diversity o...,"[('Universität Bremen', 'https://gepris.dfg.de...",[('Forschungsinstitut für Nutztierbiologie (FB...,[('Alfred-Wegener-InstitutHelmholtz-Zentrum fü...,"[('Professor Dr. Frank Oliver Glöckner', 'htt...","[('Professor Dr. Christian Ammer', 'https://g...","[('Professorin Dr. Aletta Bonn', 'https://gep...",NaN
5,https://gepris.dfg.de/gepris/projekt/442077441,DataPLANT,Biology,https://nfdi4plants.de,"In modern hypothesis-driven research, scientis...","[('Albert-Ludwigs-Universität Freiburg', 'http...","[('Eberhard Karls Universität Tübingen', 'http...",[('Helmholtz Zentrum München - Deutsches Forsc...,"[('Dr. Dirk von Suchodoletz', 'https://gepris....","[('Professor Dr. Rolf Backofen', 'https://gep...","[('Dr. Jens Krüger', 'https://gepris.dfg.de/g...",NaN
6,https://gepris.dfg.de/gepris/projekt/442146713,NFDI4Ing,"Mechanical and Industrial Engineering, Thermal...",https://nfdi4ing.de,NFDI4Ing brings together the engineering commu...,[('Rheinisch-Westfälische Technische Hochschul...,[('Deutsches Zentrum für Luft- und Raumfahrt e...,[('Bundesanstalt für Materialforschung und -pr...,"[('Professor Dr.-Ing. Robert Schmitt', 'https...","[('Professorin Dr. Jasmin Aghassi-Hagmann', '...","[('Verena Anthofer', 'https://gepris.dfg.de/g...",NaN
7,https://gepris.dfg.de/gepris/projekt/442326535,NFDI4Health,Medicine,https://www.nfdi4health.de,Germany has accumulated a wealth of health-rel...,[('Deutsche Zentralbibliothek für Medizin (ZB ...,"[('Charité - Universitätsmedizin Berlin', 'htt...",[('Behörde für Gesundheit und Verbraucherschut...,"[('Professorin Dr. Juliane Fluck', 'https://g...","[('Professor Dr. Thomas Behrens', 'https://ge...","[('Professor Dr. Wolfgang Ahrens', 'https://g...",Na

The column 'Spokesperson' contains data we need: the name of a chairperson and its GEPRIS ID.

In [3]:
for person in df['Spokesperson']:
    print(person)

[('Professor Dr. Oliver  Stegle, Ph.D.', 'https://gepris.dfg.de/gepris/person/196795178')]
[('Dr. Andreas  Förster', 'https://gepris.dfg.de/gepris/person/53182641')]
[('Professor Torsten  Schrade', 'https://gepris.dfg.de/gepris/person/40014658')]
[('Professor Dr. Christoph  Steinbeck', 'https://gepris.dfg.de/gepris/person/1438900')]
[('Professor Dr. Frank Oliver  Glöckner', 'https://gepris.dfg.de/gepris/person/2043896')]
[('Dr. Dirk von Suchodoletz', 'https://gepris.dfg.de/gepris/person/92382825')]
[('Professor Dr.-Ing. Robert  Schmitt', 'https://gepris.dfg.de/gepris/person/969363')]
[('Professorin Dr. Juliane  Fluck', 'https://gepris.dfg.de/gepris/person/1817110')]
[('Professor Dr. Christof  Wolf', 'https://gepris.dfg.de/gepris/person/1610376')]
[('Professor Dr. Erhard  Hinrichs', 'https://gepris.dfg.de/gepris/person/459920279')]
[('Professor Dr. Lars  Bernard', 'https://gepris.dfg.de/gepris/person/47182391')]
[('Professor Dr. Florian  Stahl', 'https://gepris.dfg.de/gepris/person/2743

Apart of that we need mapping between GEPRIS and Wikidata. We can query the Wikidata SPARQL endpoint. Unfortunately, some of the chairpersons did not have a corresponding page at Wikidata and majority of chairperson's Wikidata pages did not have a claim with GEPRIS ID. So we created those pages and added GEPRIS IDs to all pages of chairpersons.

Let's create a helper function `get_gepris2wikidata()` which for given GEPRIS ID returns a list with the corresponding Wikidata page URL and its label.

In [4]:
def get_gepris2wikidata(gepris, person=True):
    if person:
        prop = "P4872" # GEPRIS person ID
    else:
        prop = "P4870" # GEPRIS project ID
    query = """
    SELECT ?item ?itemLabel 
    WHERE 
    {
      ?item wdt:""" + prop + """ '""" + gepris + """'
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """
    r = requests.get('https://query.wikidata.org/sparql',
                     params = {'format': 'json', 'query': query},
                     headers = {"User-Agent": "gepris2wikidata"})
    r.raise_for_status
    results = r.json().get('results').get('bindings')
    try:
        wd_id = results[0].get('item').get('value', '')
        wd_label = results[0].get('itemLabel').get('value', '')
    except Exception:
        wd_id, wd_label = '', ''
    return [wd_id, wd_label]

Now all chairpersons have the Wikidata pages and a claim with GEPRIS ID:

In [5]:
gepris2wikidata = {}
for gepris_sub, person in zip(df['gepris'], df['Spokesperson']):
    gepris_id, gepris_name = eval(person)[0][1], eval(person)[0][0]
    [chair_wd, chair_name] = get_gepris2wikidata(eval(person)[0][1].split('/')[-1])
    gepris2wikidata[gepris_sub] = chair_wd
    print([gepris_sub], [gepris_id, gepris_name ], [chair_wd, chair_name])
print(gepris2wikidata)

['https://gepris.dfg.de/gepris/projekt/441914366'] ['https://gepris.dfg.de/gepris/person/196795178', 'Professor Dr. Oliver  Stegle, Ph.D.'] ['http://www.wikidata.org/entity/Q41049272', 'Oliver Stegle']
['https://gepris.dfg.de/gepris/projekt/441926934'] ['https://gepris.dfg.de/gepris/person/53182641', 'Dr. Andreas  Förster'] ['http://www.wikidata.org/entity/Q108652783', 'Andreas Förster']
['https://gepris.dfg.de/gepris/projekt/441958017'] ['https://gepris.dfg.de/gepris/person/40014658', 'Professor Torsten  Schrade'] ['http://www.wikidata.org/entity/Q57948193', 'Torsten Schrade']
['https://gepris.dfg.de/gepris/projekt/441958208'] ['https://gepris.dfg.de/gepris/person/1438900', 'Professor Dr. Christoph  Steinbeck'] ['http://www.wikidata.org/entity/Q5111731', 'Christoph Steinbeck']
['https://gepris.dfg.de/gepris/projekt/442032008'] ['https://gepris.dfg.de/gepris/person/2043896', 'Professor Dr. Frank Oliver  Glöckner'] ['http://www.wikidata.org/entity/Q28913667', 'Frank Oliver Glöckner']
['

All consortia have claims with GEPRIS ID as well:

In [6]:
for gepris in df['gepris']:
    print([gepris, get_gepris2wikidata(gepris.split('/')[-1], person=False)])

['https://gepris.dfg.de/gepris/projekt/441914366', ['http://www.wikidata.org/entity/Q98380337', 'GHGA']]
['https://gepris.dfg.de/gepris/projekt/441926934', ['http://www.wikidata.org/entity/Q98380342', 'NFDI4Cat']]
['https://gepris.dfg.de/gepris/projekt/441958017', ['http://www.wikidata.org/entity/Q98276929', 'NFDI4Culture']]
['https://gepris.dfg.de/gepris/projekt/441958208', ['http://www.wikidata.org/entity/Q96678459', 'NFDI4Chem']]
['https://gepris.dfg.de/gepris/projekt/442032008', ['http://www.wikidata.org/entity/Q98380341', 'NFDI4BioDiversity']]
['https://gepris.dfg.de/gepris/projekt/442077441', ['http://www.wikidata.org/entity/Q98380336', 'DataPlant']]
['https://gepris.dfg.de/gepris/projekt/442146713', ['http://www.wikidata.org/entity/Q98380344', 'NFDI4Ing']]
['https://gepris.dfg.de/gepris/projekt/442326535', ['http://www.wikidata.org/entity/Q98380343', 'NFDI4Health']]
['https://gepris.dfg.de/gepris/projekt/442494171', ['http://www.wikidata.org/entity/Q98380340', 'KonsortSWD']]
['h

Wikidata SPARQL endpoint can tell us which NFDI consortia do not have yet a claim with chairperson:

In [7]:
def get_wditems_without_chairperson():
    query = """
    SELECT ?item
    WHERE 
    {
      ?item wdt:P31 wd:Q98270496.
      FILTER NOT EXISTS { ?item wdt:P488 ?x.}
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """
    r = requests.get('https://query.wikidata.org/sparql',
                     params = {'format': 'json', 'query': query},
                     headers = {"User-Agent": "gepris2wikidata"})
    r.raise_for_status
    results = r.json().get('results').get('bindings')
    items = []
    for result in results:
        items.append(result.get('item').get('value'))
    return items
chairitems = get_wditems_without_chairperson()
chairitems

['http://www.wikidata.org/entity/Q98271443',
 'http://www.wikidata.org/entity/Q98276929',
 'http://www.wikidata.org/entity/Q98380336',
 'http://www.wikidata.org/entity/Q98380337',
 'http://www.wikidata.org/entity/Q98380342',
 'http://www.wikidata.org/entity/Q98380343',
 'http://www.wikidata.org/entity/Q98380340',
 'http://www.wikidata.org/entity/Q98380341',
 'http://www.wikidata.org/entity/Q99534506',
 'http://www.wikidata.org/entity/Q108327788',
 'http://www.wikidata.org/entity/Q108542181',
 'http://www.wikidata.org/entity/Q108542327',
 'http://www.wikidata.org/entity/Q108542373',
 'http://www.wikidata.org/entity/Q108542422',
 'http://www.wikidata.org/entity/Q108542504',
 'http://www.wikidata.org/entity/Q108542607',
 'http://www.wikidata.org/entity/Q108542637']

To create claims with chairpersons, we need information about Wikidata ID, GEPRIS ID and `start time` for chairperson. Let's use inception of a consortium as start time for chairpersons.

In [8]:
dfi = pd.read_csv("../../../data/wikidata_manual_edits.csv")
dfi

,Wikidata,Consortium,GEPRIS,City,Twitter,GitHub,YouTube,Inception,Homepage
0,http://www.wikidata.org/entity/Q98380344,NFDI4Ing,442146713,Aachen,NFDI4Ing,NaN,NaN,2020-10-01T00:00:00Z,https://nfdi4ing.de
1,http://www.wikidata.org/entity/Q99534506,NFDI4Microbiota,460129525,Cologne,nfdi4microbiota,NaN,NaN,2021-10-01T00:00:00Z,https://nfdi4microbiota.de/
2,http://www.wikidata.org/entity/Q108327788,MaRDI,460135501,Berlin,mardi4nfdi,NaN,NaN,2021-10-01T00:00:00Z,https://www.mardi4nfdi.de
3,http://www.wikidata.org/entity/Q108542181,BERD@NFDI,460037581,Mannheim,BERD_NFDI,NaN,NaN,2021-10-01T00:00:00Z,https://www.berd-nfdi.de
4,http://www.wikidata.org/entity/Q108542327,DAPHNE4NFDI,460248799,Hamburg,NaN,NaN,NaN,2021-10-01T00:00:00Z,https://www.sni-portal.de/de/daphne-nfdi/daphn...
5,http://www.wikidata.org/entity/Q108542373,FAIRmat,460197019,Berlin,FAIRmat_NFDI,NaN,NaN,2021-10-01T00:00:00Z,https://www.fair-di.eu/fairmat/fairmat_/consor...
6,http://www.wikidata.org/entity/Q108542422,NFDI4DataScience,460234259,Munich,nfdi4ds,NaN,NaN,2021-10-01T00:00:00Z,https://www.nfdi4datascience.de
7,http://www.wikidata.org/entity/Q108542504,NFDI4Earth,460036893,Dresden,nfdi4earth,NaN,NaN,2021-10-01T00:00:00Z,https://www.nfdi4earth.de
8,http://www.wikidata.org/entity/Q108542607,NFDI-MatWerk,460247524,Munich,NMatwerk,NaN,NaN,2021-10-01T00:00:00Z,https://nfdi-matwerk.de
9,http://www.wikidata.org/entity/Q108542637,PUNCH4NFDI,460248186,Hamburg,Punch4NFDI,NaN,NaN,2021-10-01T00:00:00Z,https://www.punch4nfdi.de


Now we can add the data to Wikidata using Wikidataintegrator library.

In [ ]:
from wikidata import wduser, wdpass
from wikidataintegrator import wdi_core, wdi_login

In [10]:
login = wdi_login.WDLogin(wduser, wdpass)
for wditem in chairitems:
    subdfi = dfi[ dfi['Wikidata'] == wditem ]
    inception = '+' + subdfi['Inception'].values[0]
    gepris = "https://gepris.dfg.de/gepris/projekt/" + str(subdfi['GEPRIS'].values[0])
    chair_id = gepris2wikidata[gepris].split('/')[-1]
    wd_item = wditem.split('/')[-1]
    item = wdi_core.WDItemEngine(wd_item_id=wd_item)
    qualifiers = [ wdi_core.WDTime(inception, 'P580', is_qualifier=True, precision=9) ]
    references = [[ wdi_core.WDUrl(value=gepris, prop_nr='P854', is_reference=True) ]]
    statement = [ wdi_core.WDItemID(chair_id, prop_nr='P488', qualifiers=qualifiers, references=references) ]
    item.update(data=statement)
    print(wd_item, chair_id, gepris, inception)
    item.write(login, bot_account=False)


Q98271443 Q51843931 https://gepris.dfg.de/gepris/projekt/460033370 +2021-10-01T00:00:00Z
Q98276929 Q57948193 https://gepris.dfg.de/gepris/projekt/441958017 +2020-10-01T00:00:00Z
Q98380336 Q97060913 https://gepris.dfg.de/gepris/projekt/442077441 +2020-10-01T00:00:00Z
Q98380337 Q41049272 https://gepris.dfg.de/gepris/projekt/441914366 +2020-10-01T00:00:00Z
Q98380342 Q108652783 https://gepris.dfg.de/gepris/projekt/441926934 +2020-10-01T00:00:00Z
Q98380343 Q50419539 https://gepris.dfg.de/gepris/projekt/442326535 +2020-10-01T00:00:00Z
Q98380340 Q31773505 https://gepris.dfg.de/gepris/projekt/442494171 +2020-10-01T00:00:00Z
Q98380341 Q28913667 https://gepris.dfg.de/gepris/projekt/442032008 +2020-10-01T00:00:00Z
Q99534506 Q18744528 https://gepris.dfg.de/gepris/projekt/460129525 +2021-10-01T00:00:00Z
Q108327788 Q83187047 https://gepris.dfg.de/gepris/projekt/460135501 +2021-10-01T00:00:00Z
Q108542181 Q108652977 https://gepris.dfg.de/gepris/projekt/460037581 +2021-10-01T00:00:00Z
Q108542327 Q28359